# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [11]:
!pip3 install pickle5
import pandas as pd
import pickle5 as pickle
f = open("data_aug_random_deletion.pkl","rb")
df = pickle.load(f)


In [12]:
df

,sentence,labels
0,The Rock is destined to be the 21st Century 's...,16
1,The gorgeously elaborate continuation of `` Th...,19
2,Effective but too-tepid biopic,12
3,If you sometimes like to go to the movies to h...,17
4,"Emerges as something rare , an issue movie tha...",20
...,...,...
20220,ever in DiCaprio 's is performance best anythi...,24
20221,that as and telling story that examines forbid...,24
20222,Jeffrey the 's performance exterminator Tambor...,24
20223,"Twenty remains years its first release , E.T. ...",24


In [13]:
df.shape

(20225, 2)

In [14]:
df.labels.value_counts()

19    953
7     853
16    819
15    800
21    800
1     800
17    800
2     800
18    800
3     800
4     800
20    800
5     800
6     800
14    800
22    800
23    800
8     800
24    800
9     800
10    800
11    800
12    800
13    800
0     800
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [15]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [16]:
Sentence = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [17]:
fields = [('sentence', Sentence),('labels',Label)]

In [8]:
df.iloc[11289]

sentence    ... the story is far-flung , illogical , and p...
labels                                                      0
Name: 11289, dtype: object

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [18]:
example = [torchtext.legacy.data.Example.fromlist([df.sentence[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [19]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

stanfordDataset = torchtext.legacy.data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [20]:
(train, valid) = stanfordDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [21]:
(len(train), len(valid))

(17191, 3034)

An example from the dataset:

In [22]:
vars(train.examples[100])

{'labels': 8,
 'sentence': ['A',
  'sophomoric',
  'exploration',
  'of',
  '`',
  'life',
  'problems',
  "'",
  'most',
  'people',
  'solved',
  'long',
  'ago',
  '--',
  'or',
  'at',
  'least',
  'got',
  'tired',
  'of',
  'hearing',
  'people',
  'kvetch',
  'about',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [23]:
Sentence.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [15]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18400
Size of label vocab :  25
Top 10 words appreared repeatedly : [('.', 15990), (',', 14263), ('the', 12036), ('of', 9030), ('and', 8574), ('a', 8422), ('to', 5736), ('-', 5251), ('is', 4925), ("'s", 4847)]
Labels :  defaultdict(None, {19: 0, 7: 1, 11: 2, 1: 3, 4: 4, 16: 5, 17: 6, 0: 7, 9: 8, 15: 9, 21: 10, 5: 11, 6: 12, 2: 13, 14: 14, 24: 15, 10: 16, 20: 17, 8: 18, 23: 19, 3: 20, 13: 21, 22: 22, 12: 23, 18: 24})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [18]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [19]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [20]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 25
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [21]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18400, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=25, bias=True)
)
The model has 5,764,125 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [22]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [23]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.sentence   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [24]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [26]:
N_EPOCHS = 50
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 2.449 | Train Acc: 84.03%
	 Val. Loss: 2.783 |  Val. Acc: 50.71% 

	Train Loss: 2.445 | Train Acc: 84.39%
	 Val. Loss: 2.781 |  Val. Acc: 50.68% 

	Train Loss: 2.442 | Train Acc: 84.61%
	 Val. Loss: 2.783 |  Val. Acc: 50.39% 

	Train Loss: 2.439 | Train Acc: 84.87%
	 Val. Loss: 2.781 |  Val. Acc: 50.74% 

	Train Loss: 2.435 | Train Acc: 85.22%
	 Val. Loss: 2.780 |  Val. Acc: 50.55% 

	Train Loss: 2.429 | Train Acc: 86.08%
	 Val. Loss: 2.782 |  Val. Acc: 50.42% 

	Train Loss: 2.423 | Train Acc: 86.67%
	 Val. Loss: 2.784 |  Val. Acc: 50.05% 

	Train Loss: 2.418 | Train Acc: 87.20%
	 Val. Loss: 2.780 |  Val. Acc: 50.65% 

	Train Loss: 2.413 | Train Acc: 87.62%
	 Val. Loss: 2.781 |  Val. Acc: 50.50% 

	Train Loss: 2.410 | Train Acc: 87.85%
	 Val. Loss: 2.779 |  Val. Acc: 50.82% 

	Train Loss: 2.406 | Train Acc: 88.17%
	 Val. Loss: 2.780 |  Val. Acc: 50.51% 

	Train Loss: 2.403 | Train Acc: 88.35%
	 Val. Loss: 2.779 |  Val. Acc: 50.82% 

	Train Loss: 2.402 | Train Acc: 88.44%
	

## Model Testing

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [102]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [28]:
def random_deletion(words, p=0.1): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [136]:
### checking random_deletion

t =random_deletion(["I" ,"am", "just" ,"checking", "how", "this" ,"works "])
t

['am', 'just', 'checking', 'how', 'this', 'works ']

In [27]:
def use_random_deletion(sentence):
  word_list = sentence.split()
  new_word_list=random_deletion(word_list)
  new_sentence = ' '.join(new_word_list)
  return new_sentence

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [25]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [26]:
def use_random_swap(sentence):
  word_list = sentence.split()
  new_word_list=random_swap(word_list)
  new_sentence = ' '.join(new_word_list)
  return new_sentence




For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [37]:
!pip install googletrans==3.1.0a0   # this is alpha version, default version was giving error.  error 'NoneType' object has no attribute 'group'
import random
import googletrans
#import googletrans.Translator
from googletrans import Translator
translator = Translator()


def use_back_translation(sentence):
        sentence = [sentence]

        available_langs = list(googletrans.LANGUAGES.keys()) 
        trans_lang = random.choice(available_langs) 
        print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

        #translations = translator.translate("hello", dest=trans_lang) 
        translations = translator.translate(sentence,  dest=trans_lang)

        t_text = [t.text for t in translations]
        print(t_text)

        translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
        en_text = [t.text for t in translations_en_random]
        #print(en_text, " *********************** ")
        return(en_text[0])

In [39]:
use_back_translation("what is your name?")

Translating to luxembourgish
['what is your name?']


'what is your name?'

In [40]:
####    Augmenting the dataset

for label_class in range(len(Label.vocab)):
     df_aug=df[df['labels']==label_class]
     old_num_samples=df_aug.shape[0]
     print(old_num_samples)
     #new_num_samples=600
     new_num_samples=1500

     if new_num_samples > old_num_samples:
        for aug_sample in range(new_num_samples-old_num_samples):
              
              random_old_sample_no=random.randint(0,old_num_samples-1)
              print(random_old_sample_no, "oopiok")
              
              df_new=df_aug.iloc[random_old_sample_no]

              #  using random sample
              #new_sentence=use_random_swap(df_aug.iloc[random_old_sample_no,0])  
              #df_new.iloc[0]=new_sentence

              #  using random deletion
              #new_sentence=use_random_deletion(df_aug.iloc[random_old_sample_no,0])  
              #df_new.iloc[0]=new_sentence

              #  using back translation
              new_sentence=use_back_translation(df_aug.iloc[random_old_sample_no,0])  
              df_new.iloc[0]=new_sentence

              df=df.append(df_new,ignore_index = True)

791 oopiok
Translating to japanese
['Make like dodge this and one title the .']
345 oopiok
Translating to hindi
['Behind the glitz , Hollywood is sordid and disgusting .']
134 oopiok
Translating to serbian
['An amateurish , quasi-improvised acting exercise shot on ugly digital video .']
139 oopiok
Translating to sindhi
['A limp Eddie Murphy vehicle that even he seems embarrassed to be part of .']
3 oopiok
Translating to afrikaans
["Many went to see the attraction for the sole reason that it was hot outside and there was air conditioning inside , and I do n't think that A.C. will help this movie one bit ."]
86 oopiok
Translating to kyrgyz
["... a plotline that 's as lumpy as two-day old porridge ... the filmmakers ' paws , sad to say , were all over this `` un-bear-able '' project !"]
317 oopiok
Translating to cebuano
['Manipulative claptrap , a period-piece movie-of-the-week , plain old blarney ... take your pick .']
168 oopiok
Translating to indonesian
["Godard 's ode to tackling life

KeyboardInterrupt: ignored

In [32]:
df.labels.value_counts()

19    953
7     853
16    819
15    800
21    800
1     800
17    800
2     800
18    800
3     800
4     800
20    800
5     800
6     800
14    800
22    800
23    800
8     800
24    800
9     800
10    800
11    800
12    800
13    800
0     800
Name: labels, dtype: int64

In [33]:
df.to_pickle("data_aug_random_deletion.pkl")

In [34]:
df

,sentence,labels
0,The Rock is destined to be the 21st Century 's...,16
1,The gorgeously elaborate continuation of `` Th...,19
2,Effective but too-tepid biopic,12
3,If you sometimes like to go to the movies to h...,17
4,"Emerges as something rare , an issue movie tha...",20
...,...,...
20220,ever in DiCaprio 's is performance best anythi...,24
20221,that as and telling story that examines forbid...,24
20222,Jeffrey the 's performance exterminator Tambor...,24
20223,"Twenty remains years its first release , E.T. ...",24
